<a href="https://colab.research.google.com/github/atanasiusps/Machine-Learning-Enrichment_Task/blob/main/Chapter-13/Chapter_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CODE REPRODUCTION

Import Library

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if IS_COLAB or IS_KAGGLE:
    %pip install -q -U tfx
    print("You can safely ignore the package incompatibility errors.")

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "data"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 1.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
You can safely ignore the package incompatibility errors.


In [ ]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [ ]:
dataset = tf.data.Dataset.range(10)

In [ ]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [ ]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [ ]:
dataset = dataset.map(lambda x: x * 2)

In [ ]:
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int64)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int64)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int64)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int64)
tf.Tensor([16 18], shape=(2,), dtype=int64)


In [ ]:
#dataset = dataset.apply(tf.data.experimental.unbatch()) # Now deprecated
dataset = dataset.unbatch()

In [ ]:
dataset = dataset.filter(lambda x: x < 10)  # keep only items < 10

In [ ]:
for item in dataset.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)


In [ ]:
tf.random.set_seed(42)

dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=3, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([1 3 0 4 2 5 6], shape=(7,), dtype=int64)
tf.Tensor([8 7 1 0 3 2 5], shape=(7,), dtype=int64)
tf.Tensor([4 6 9 8 9 7 0], shape=(7,), dtype=int64)
tf.Tensor([3 1 4 5 2 8 7], shape=(7,), dtype=int64)
tf.Tensor([6 9], shape=(2,), dtype=int64)


Split the California dataset to multiple CSV files

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

In [ ]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

In [ ]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [ ]:
import pandas as pd

pd.read_csv(train_filepaths[0]).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,np.float64(3.5214),np.float64(15.0),np.float64(3.0499445061043287),np.float64(1.106548279689234),np.float64(1447.0),np.float64(1.6059933407325193),np.float64(37.63),np.float64(-122.43),np.float64(1.442)
1,np.float64(5.3275),np.float64(5.0),np.float64(6.490059642147117),np.float64(0.9910536779324056),np.float64(3464.0),np.float64(3.4433399602385686),np.float64(33.69),np.float64(-117.39),np.float64(1.687)
2,np.float64(3.1),np.float64(29.0),np.float64(7.5423728813559325),np.float64(1.5915254237288134),np.float64(1328.0),np.float64(2.2508474576271187),np.float64(38.44),np.float64(-122.98),np.float64(1.621)
3,np.float64(7.1736),np.float64(12.0),np.float64(6.289002557544757),np.float64(0.9974424552429667),np.float64(1054.0),np.float64(2.6956521739130435),np.float64(33.55),np.float64(-117.7),np.float64(2.621)
4,np.float64(2.0549),np.float64(13.0),np.float64(5.312457454050374),np.float64(1.0850918992511913),np.float64(3297.0),np.float64(2.2443839346494214),np.float64(33.93),np.float64(-116.93),np.float64(0.956)


In [ ]:
with open(train_filepaths[0]) as f:
    for i in range(5):
        print(f.readline(), end="")

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
np.float64(3.5214),np.float64(15.0),np.float64(3.0499445061043287),np.float64(1.106548279689234),np.float64(1447.0),np.float64(1.6059933407325193),np.float64(37.63),np.float64(-122.43),np.float64(1.442)
np.float64(5.3275),np.float64(5.0),np.float64(6.490059642147117),np.float64(0.9910536779324056),np.float64(3464.0),np.float64(3.4433399602385686),np.float64(33.69),np.float64(-117.39),np.float64(1.687)
np.float64(3.1),np.float64(29.0),np.float64(7.5423728813559325),np.float64(1.5915254237288134),np.float64(1328.0),np.float64(2.2508474576271187),np.float64(38.44),np.float64(-122.98),np.float64(1.621)
np.float64(7.1736),np.float64(12.0),np.float64(6.289002557544757),np.float64(0.9974424552429667),np.float64(1054.0),np.float64(2.6956521739130435),np.float64(33.55),np.float64(-117.7),np.float64(2.621)


In [ ]:
train_filepaths

['datasets/housing/my_train_00.csv',
 'datasets/housing/my_train_01.csv',
 'datasets/housing/my_train_02.csv',
 'datasets/housing/my_train_03.csv',
 'datasets/housing/my_train_04.csv',
 'datasets/housing/my_train_05.csv',
 'datasets/housing/my_train_06.csv',
 'datasets/housing/my_train_07.csv',
 'datasets/housing/my_train_08.csv',
 'datasets/housing/my_train_09.csv',
 'datasets/housing/my_train_10.csv',
 'datasets/housing/my_train_11.csv',
 'datasets/housing/my_train_12.csv',
 'datasets/housing/my_train_13.csv',
 'datasets/housing/my_train_14.csv',
 'datasets/housing/my_train_15.csv',
 'datasets/housing/my_train_16.csv',
 'datasets/housing/my_train_17.csv',
 'datasets/housing/my_train_18.csv',
 'datasets/housing/my_train_19.csv']

Building an Input Pipeline

In [ ]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)
for filepath in filepath_dataset:
    print(filepath)

tf.Tensor(b'datasets/housing/my_train_15.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_08.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_03.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_04.csv', shape=(), dtype=string)
tf.Ten

In [ ]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

In [ ]:
for line in dataset.take(5):
    print(line.numpy())

b'np.float64(4.6477),np.float64(38.0),np.float64(5.03728813559322),np.float64(0.911864406779661),np.float64(745.0),np.float64(2.5254237288135593),np.float64(32.64),np.float64(-117.07),np.float64(1.504)'
b'np.float64(8.72),np.float64(44.0),np.float64(6.163179916317992),np.float64(1.0460251046025104),np.float64(668.0),np.float64(2.794979079497908),np.float64(34.2),np.float64(-118.18),np.float64(4.159)'
b'np.float64(3.8456),np.float64(35.0),np.float64(5.461346633416459),np.float64(0.9576059850374065),np.float64(1154.0),np.float64(2.8778054862842892),np.float64(37.96),np.float64(-122.05),np.float64(1.598)'
b'np.float64(3.3456),np.float64(37.0),np.float64(4.514084507042254),np.float64(0.9084507042253521),np.float64(458.0),np.float64(3.2253521126760565),np.float64(36.67),np.float64(-121.7),np.float64(2.526)'
b'np.float64(3.6875),np.float64(44.0),np.float64(4.524475524475524),np.float64(0.993006993006993),np.float64(457.0),np.float64(3.195804195804196),np.float64(34.04),np.float64(-118.15),np

In [ ]:
record_defaults=[0, np.nan, tf.constant(np.nan, dtype=tf.float64), "Hello", tf.constant([])]
parsed_fields = tf.io.decode_csv('1,2,3,4,5', record_defaults)
parsed_fields

[<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.0>,
 <tf.Tensor: shape=(), dtype=float64, numpy=3.0>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'4'>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]

In [ ]:
parsed_fields = tf.io.decode_csv(',,,,5', record_defaults)
parsed_fields

[<tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float64, numpy=nan>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'Hello'>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]

In [ ]:
try:
    parsed_fields = tf.io.decode_csv(',,,,', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

{{function_node __wrapped__DecodeCSV_OUT_TYPE_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} Field 4 is required but missing in record 0! [Op:DecodeCSV]


In [ ]:
try:
    parsed_fields = tf.io.decode_csv('1,2,3,4,5,6,7', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

{{function_node __wrapped__DecodeCSV_OUT_TYPE_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} Expect 5 fields but have 7 in record 0 [Op:DecodeCSV]


In [ ]:
n_inputs = 8 # X_train.shape[-1]

@tf.function
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

In [ ]:
preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 0.16579157,  1.216324  , -0.05204565, -0.39215982, -0.5277444 ,
        -0.26334876,  0.8543047 , -1.3072057 ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.782], dtype=float32)>)

In [ ]:
def csv_reader_dataset(filepaths, batch_size):
    def preprocess(line):
        defaults = [0.0, 0.0, 0.0, 0.0, 0]  # misal 4 fitur float + 1 label int
        fields = tf.io.decode_csv(line, record_defaults=defaults)
        features = tf.stack(fields[:-1])
        label = fields[-1]
        return features, label

    dataset = tf.data.TextLineDataset(filepaths)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch_size)
    return dataset


In [ ]:
# Menampilkan beberapa baris pertama dari file CSV
def tampilkan_isi_csv(filepath, n=5):
    with open(filepath, 'r') as f:
        for i in range(n):
            line = f.readline().strip()
            print(f"Baris {i+1}: {line}")

# Contoh penggunaan
tampilkan_isi_csv(train_filepaths[0])


Baris 1: MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
Baris 2: np.float64(3.5214),np.float64(15.0),np.float64(3.0499445061043287),np.float64(1.106548279689234),np.float64(1447.0),np.float64(1.6059933407325193),np.float64(37.63),np.float64(-122.43),np.float64(1.442)
Baris 3: np.float64(5.3275),np.float64(5.0),np.float64(6.490059642147117),np.float64(0.9910536779324056),np.float64(3464.0),np.float64(3.4433399602385686),np.float64(33.69),np.float64(-117.39),np.float64(1.687)
Baris 4: np.float64(3.1),np.float64(29.0),np.float64(7.5423728813559325),np.float64(1.5915254237288134),np.float64(1328.0),np.float64(2.2508474576271187),np.float64(38.44),np.float64(-122.98),np.float64(1.621)
Baris 5: np.float64(7.1736),np.float64(12.0),np.float64(6.289002557544757),np.float64(0.9974424552429667),np.float64(1054.0),np.float64(2.6956521739130435),np.float64(33.55),np.float64(-117.7),np.float64(2.621)


In [ ]:
def bersihkan_csv(input_path, output_path):
    with open(input_path, 'r') as fin, open(output_path, 'w') as fout:
        for line in fin:
            if line.strip().startswith("MedInc"):
                continue  # hapus header
            cleaned = line.replace("np.float64(", "").replace(")", "")
            fout.write(cleaned)


In [ ]:
# Cell 1: Import dan fungsi pembaca dataset
import tensorflow as tf

def csv_reader_dataset(filepaths, batch_size=32, repeat=None, shuffle_buffer_size=1000):
    def preprocess(line):
        # Sesuaikan jumlah kolom (contoh 9 kolom: 8 fitur + 1 label)
        defaults = [0.0] * 9
        fields = tf.io.decode_csv(line, record_defaults=defaults)
        features = tf.stack(fields[:-1])  # semua kecuali kolom terakhir sebagai fitur
        label = fields[-1]  # kolom terakhir sebagai label
        return features, label

    dataset = tf.data.TextLineDataset(filepaths)
    dataset = dataset.skip(1)  # skip header kalau ada
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    if repeat:
        dataset = dataset.repeat(repeat)

    dataset = dataset.shuffle(shuffle_buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


In [ ]:
# Cell 2: Tentukan file path CSV (pastikan file sudah bersih tanpa header)
train_filepaths = ["train_clean.csv"]
valid_filepaths = ["valid_clean.csv"]


In [ ]:
# Cell 3: Buat dataset train dan validasi
batch_size = 32
train_set = csv_reader_dataset(train_filepaths, batch_size=batch_size, repeat=-1)
valid_set = csv_reader_dataset(valid_filepaths, batch_size=batch_size)


In [ ]:
# Cell 4: Siapkan model sederhana
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation='relu', input_shape=(8,)),
    tf.keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Cell 5: Tentukan jumlah data training dan hitung steps per epoch
num_train_samples = 20640  # ganti sesuai data kamu
steps_per_epoch = num_train_samples // batch_size


In [ ]:
import os
print(os.listdir())


['.config', 'train_clean.csv', 'datasets', 'images', 'sample_data']


In [ ]:
from google.colab import files
files.upload()  # lalu pilih file valid.csv dari komputer kamu


{}

In [ ]:
def remove_header(input_path, output_path):
    with open(input_path, 'r') as fin, open(output_path, 'w') as fout:
        next(fin)  # skip header
        for line in fin:
            fout.write(line)

# Contoh buat valid_clean.csv
remove_header("valid.csv", "valid_clean.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'valid.csv'

# BEBERAPA LIBRARY TIDAK BISA DIJALANKAN SEHINGGA TIDAK DAPAT DILANJUTKAN